In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(num_jobs,verbose,url,save_file,newData = True):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="../../../chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    driver.get(url)
    (driver.page_source).encode('utf-8','ignore')
    
    
    columns =  ["Position",
                "Description",
                "Company",
                "Location",
                "Glassdoor Salary Estimate",
                "Rating",
                "Size",
                "Revenue",
                "Year Founded",
                "Industry",
                "Sector"]

    count = 0

    while count < num_jobs:  #If true, should be still looking for new jobs.
        
        jobs = []

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("modal_closeIcon").click()   #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:
            count +=1

            print("Progress: {}".format("" + str(count) + "/" + str(num_jobs)))
            if count >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Position" : job_title,
            "Description" : job_description,
            "Company" : company_name.split('\n')[0],
            "Location" : location,
            "Glassdoor Salary Estimate" : salary_estimate,
            "Rating" : rating,
            "Size" : size,
            "Revenue" : revenue,
            "Year Founded" : founded,
            "Industry" : industry,
            "Sector" : sector})

        newJobs = pd.DataFrame(jobs)
        if newData:
            newJobs.to_csv(save_file)
            newData = False
        else:
            oldJobs = pd.read_csv(save_file)
            allJobs = pd.concat([oldJobs,newJobs])
            allJobs.to_csv(save_file)
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
    return

In [3]:
url1 = 'https://www.glassdoor.ca/Job/canada-senior-data-scientist-jobs-SRCH_IL.0,6_IN3_KO7,28_IP1.htm'
dataFile1 = r'seniorCanadaPostings2.csv'
get_jobs(900, False,url1,dataFile1,newData = True)

Progress: 1/900
Progress: 2/900
Progress: 3/900
Progress: 4/900
Progress: 5/900
Progress: 6/900
Progress: 7/900
Progress: 8/900
Progress: 9/900
Progress: 10/900
Progress: 11/900
Progress: 12/900
Progress: 13/900
Progress: 14/900
Progress: 15/900
Progress: 16/900
Progress: 17/900
Progress: 18/900
Progress: 19/900
Progress: 20/900
Progress: 21/900
Progress: 22/900
Progress: 23/900
Progress: 24/900
Progress: 25/900
Progress: 26/900
Progress: 27/900
Progress: 28/900
Progress: 29/900
Progress: 30/900
Progress: 31/900
Progress: 32/900
Progress: 33/900
Progress: 34/900
Progress: 35/900
Progress: 36/900
Progress: 37/900
Progress: 38/900
Progress: 39/900
Progress: 40/900
Progress: 41/900
Progress: 42/900
Progress: 43/900
Progress: 44/900
Progress: 45/900
Progress: 46/900
Progress: 47/900
Progress: 48/900
Progress: 49/900
Progress: 50/900
Progress: 51/900
Progress: 52/900
Progress: 53/900
Progress: 54/900
Progress: 55/900
Progress: 56/900
Progress: 57/900
Progress: 58/900
Progress: 59/900
Progre

C:\Users\Jack Ellis\anaconda3\envs\jackDopl\lib\site-packages\ipykernel_launcher.py:162: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Progress: 62/900
Progress: 63/900
Progress: 64/900
Progress: 65/900
Progress: 66/900
Progress: 67/900
Progress: 68/900
Progress: 69/900
Progress: 70/900
Progress: 71/900
Progress: 72/900
Progress: 73/900
Progress: 74/900
Progress: 75/900
Progress: 76/900
Progress: 77/900
Progress: 78/900
Progress: 79/900
Progress: 80/900
Progress: 81/900
Progress: 82/900
Progress: 83/900
Progress: 84/900
Progress: 85/900
Progress: 86/900
Progress: 87/900
Progress: 88/900
Progress: 89/900
Progress: 90/900
Progress: 91/900
Progress: 92/900
Progress: 93/900
Progress: 94/900
Progress: 95/900
Progress: 96/900
Progress: 97/900
Progress: 98/900
Progress: 99/900
Progress: 100/900
Progress: 101/900
Progress: 102/900
Progress: 103/900
Progress: 104/900
Progress: 105/900
Progress: 106/900
Progress: 107/900
Progress: 108/900
Progress: 109/900
Progress: 110/900
Progress: 111/900
Progress: 112/900
Progress: 113/900
Progress: 114/900
Progress: 115/900
Progress: 116/900
Progress: 117/900
Progress: 118/900
Progress: 11

Progress: 520/900
Progress: 521/900
Progress: 522/900
Progress: 523/900
Progress: 524/900
Progress: 525/900
Progress: 526/900
Progress: 527/900
Progress: 528/900
Progress: 529/900
Progress: 530/900
Progress: 531/900
Progress: 532/900
Progress: 533/900
Progress: 534/900
Progress: 535/900
Progress: 536/900
Progress: 537/900
Progress: 538/900
Progress: 539/900
Progress: 540/900
Progress: 541/900
Progress: 542/900
Progress: 543/900
Progress: 544/900
Progress: 545/900
Progress: 546/900
Progress: 547/900
Progress: 548/900
Progress: 549/900
Progress: 550/900
Progress: 551/900
Progress: 552/900
Progress: 553/900
Progress: 554/900
Progress: 555/900
Progress: 556/900
Progress: 557/900
Progress: 558/900
Progress: 559/900
Progress: 560/900
Progress: 561/900
Progress: 562/900
Progress: 563/900
Progress: 564/900
Progress: 565/900
Progress: 566/900
Progress: 567/900
Progress: 568/900
Progress: 569/900
Progress: 570/900
Progress: 571/900
Progress: 572/900
Progress: 573/900
Progress: 574/900
Progress: 

In [3]:
url2 = 'https://www.glassdoor.ca/Job/canada-junior-data-scientist-jobs-SRCH_IL.0,6_IN3_KO7,28_IP9.htm'
dataFile2 = r'juniorCanadaPostings2.csv'
get_jobs(180, False,url2,dataFile2,newData=False)

Progress: 1/180
Progress: 2/180
Progress: 3/180
Progress: 4/180
Progress: 5/180
Progress: 6/180
Progress: 7/180
Progress: 8/180
Progress: 9/180
Progress: 10/180
Progress: 11/180
Progress: 12/180
Progress: 13/180
Progress: 14/180
Progress: 15/180
Progress: 16/180
Progress: 17/180
Progress: 18/180
Progress: 19/180
Progress: 20/180
Progress: 21/180
Progress: 22/180
Progress: 23/180
Progress: 24/180
Progress: 25/180
Progress: 26/180
Progress: 27/180
Progress: 28/180
Progress: 29/180
Progress: 30/180


C:\Users\Jack Ellis\anaconda3\envs\jackDopl\lib\site-packages\ipykernel_launcher.py:161: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Progress: 31/180
Progress: 32/180
Progress: 33/180
Progress: 34/180
Progress: 35/180
Progress: 36/180
Progress: 37/180
Progress: 38/180
Progress: 39/180
Progress: 40/180
Progress: 41/180
Progress: 42/180
Progress: 43/180
Progress: 44/180
Progress: 45/180
Progress: 46/180
Progress: 47/180
Progress: 48/180
Progress: 49/180
Progress: 50/180
Progress: 51/180
Progress: 52/180
Progress: 53/180
Progress: 54/180
Progress: 55/180
Progress: 56/180
Progress: 57/180
Progress: 58/180
Progress: 59/180
Progress: 60/180
Progress: 61/180
Progress: 62/180
Progress: 63/180
Progress: 64/180
Progress: 65/180
Progress: 66/180
Progress: 67/180
Progress: 68/180
Progress: 69/180
Progress: 70/180
Progress: 71/180
Progress: 72/180
Progress: 73/180
Progress: 74/180
Progress: 75/180
Progress: 76/180
Progress: 77/180
Progress: 78/180
Progress: 79/180
Progress: 80/180
Progress: 81/180
Progress: 82/180
Progress: 83/180
Progress: 84/180
Progress: 85/180
Progress: 86/180
Progress: 87/180
Progress: 88/180
Progress: 89/1

In [ ]:
url3 = 'https://www.glassdoor.ca/Job/uk-junior-data-scientist-jobs-SRCH_IL.0,2_IN2_KO3,24.htm'
dataFile3 = r'juniorUKPostings.csv'
get_jobs(210, False,url3,dataFile3)

In [ ]:
url4 = 'https://www.glassdoor.ca/Job/uk-junior-data-scientist-jobs-SRCH_IL.0,2_IN2_KO3,24.htm'
dataFile4 = r'seniorUKPostings.csv'
get_jobs(900, False,url4,dataFile4)